<a href="https://colab.research.google.com/github/ben854719/Arctic-Sentinel-AI-Native-ISR-Dashboard/blob/main/Agentic_AI_Assistant_Iamge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph
!pip install langsmith

  Using cached langchain_google_genai-3.0.3-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_core-1.0.5-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (

In [3]:
import os
os.environ["LANGSMITH_API_KEY"] = "LangSmith"

In [4]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [5]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
sample_data


In [6]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
from langgraph.graph import StateGraph, END
from IPython.display import display, HTML
from typing import TypedDict
import os
import random
import json
from mcp.server.fastmcp import FastMCP
from google.colab import userdata
from langsmith import traceable
from google.colab import userdata

# Import APl Key.
api_key = userdata.get("Ben856")
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")
os.environ["GOOGLE_API_KEY"] = api_key

# Agent State Definition.

class AgentState(TypedDict):
   input: str
   output: str
   trace_id: str
   context: dict

# Prompt Template.

prompt = ChatPromptTemplate.from_template("""
You are a strategic AI agent assisting with the development of Arctic Sentinel—an AI-native ISR dashboard for Arctic vulnerability mapping.
Respond with modular, secure, and expressive outputs tailored for employer engagement and demo impact. Input: {input}
""")

# Tools Definition.

def telemetry_tool(input: str) -> str:
    return f"Telemetry captured for input: {input}"

def overlay_tool(input: str) -> str:
    return f"Overlay captured for input: {input}"

def arcgis_tool(input: str) -> str:
    map_url = "https://www.arcgis.com/apps/mapviewer/index.html?webmap=6d8144399610426e8f57246e2607782d"
    return f"View Arctic vulnerability map: <a href='{map_url}' target='_blank'>ArcGIS Viewer</a>"

tools = [
    Tool.from_function(name="telemetry", func=telemetry_tool, description="Secure telemetry capture"),
    Tool.from_function(name="overlay", func=overlay_tool, description="Narrative overlay generation"),
    Tool.from_function(name="arcgis", func=arcgis_tool, description="Link to Arctic vulnerability map")
]

# Initialize Gemini Model.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)

# LangGrap nodes.
@traceable(name="entry_node")
def entry_node(state: AgentState) -> AgentState:
    return {**state, "trace_id": f"trace-{random.randint(1000,9999)}"}

@traceable(name="llm_node")
def llm_node(state: AgentState) -> AgentState:
    response = llm.invoke(prompt.format(input=state["input"]))
    return {**state, "output": response.content}

@traceable(name="tool_node")
def tool_node(state: AgentState) -> AgentState:
    telemetry = telemetry_tool(state["input"])
    overlay = overlay_tool(state["input"])
    arcgis = arcgis_tool(state["input"])
    return {**state, "context": {
        "telemetry": telemetry,
        "overlay": overlay,
        "arcgis": arcgis
    }}

@traceable(name="display_node")
def display_node(state: AgentState) -> AgentState:
    html = f"""
    <div style='font-family:sans-serif;'>
        <h2>Agent Output</h2>
        <p><strong>Response:</strong> {state['output']}</p>
        <p><strong>Telemetry:</strong> {state['context']['telemetry']}</p>
        <p><strong>Overlay:</strong> {state['context']['overlay']}</p>
        <p><strong>Map Viewer:</strong> {state['context']['arcgis']}</p>
        <p><strong>Trace ID:</strong> {state['trace_id']}</p>
    </div>
    """
    display(HTML(html))
    return state

# LangGraph workflow.
graph = StateGraph(AgentState)
graph.add_node("entry", entry_node)
graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)
graph.add_node("display", display_node)

graph.set_entry_point("entry")
graph.add_edge("entry", "llm")
graph.add_edge("llm", "tools")
graph.add_edge("tools", "display")
graph.add_edge("display", END)

# Compile app.
agent_executor = graph.compile()

# Run agent.
agent_executor.invoke({"input": "Generate a modular demo script for Arctic Sentinel, tailored for stakeholder engagement and technical impact"})

{'input': 'Generate a modular demo script for Arctic Sentinel, tailored for stakeholder engagement and technical impact',
 'output': 'Okay, here is a modular, secure, and expressive demo script for Arctic Sentinel, designed to maximize employer engagement and technical impact.\n\n---\n\n## Arctic Sentinel: AI-Native ISR Dashboard Demo Script\n\n**Target Audience:** Strategic Stakeholders, Technical Leads, Potential Investors\n**Goal:** Showcase Arctic Sentinel\'s strategic value, technical sophistication, modular architecture, and robust security.\n**Duration:** ~15-20 minutes (flexible, with optional deep-dives)\n\n---\n\n### **Module 1: Strategic Imperative & Vision (The "Why")**\n*(Presenter: Sets the stage, passionate and authoritative)*\n\n**[0:00 - 2:00]**\n\n**Presenter:** "Good morning/afternoon. We are here today to introduce you to **Arctic Sentinel** – our vision for securing the future of the Arctic. The Arctic is no longer a remote frontier; it\'s a rapidly evolving geopol